In [10]:
import pandas as pd
import numpy as np
import datetime
from epiweeks import Week
import epiweeks
from collections import defaultdict
from pathlib import Path
import requests
import os
from datetime import datetime
from datetime import date, timedelta
from sodapy import Socrata
import warnings

import covidcast
from delphi_epidata import Epidata

covidcast.use_api_key("4bee67d2520898")


warnings.filterwarnings('ignore')

In [ ]:
# try to add in Delphi versioned data for categorical ensemble

In [43]:
TODAY = datetime.now()

# Convert dates to epiweeks
start_week = Week.fromdate(date(2025, 10, 1))
end_week = Week.fromdate(TODAY)

start_epiweek = int(f"{start_week.year}{start_week.week:02d}")
end_epiweek = int(f"{end_week.year}{end_week.week:02d}")

result_adm = Epidata.covidcast(data_source='nhsn', signals='confirmed_admissions_flu_ew_prelim',
    time_type='week', geo_type='state', time_values=Epidata.range(start_epiweek, end_epiweek), geo_value='*',
                              issues='*')
result_adm_us = Epidata.covidcast(data_source='nhsn', signals='confirmed_admissions_flu_ew_prelim',
    time_type='week', geo_type='nation', time_values=Epidata.range(start_epiweek, end_epiweek), geo_value='*',
                              issues='*')

# Convert to DataFrame
if result_adm['result'] == 1:
    dfadm = pd.DataFrame(result_adm['epidata'])
    dfadm_us = pd.DataFrame(result_adm_us['epidata'])
    #print(dfadm[['time_value', 'value', 'issue']])
else:
    print(f"No results: {result_adm.get('message')}")

dfadm = dfadm[['geo_value', 'time_value','issue','value']]
dfadm_us = dfadm_us[['geo_value', 'time_value','issue','value']]


df = pd.concat([dfadm, dfadm_us])

df['issue_date'] = df['issue'].apply(lambda x: Week(x//100, x %100).enddate())
df['target_end_date'] = df['time_value'].apply(lambda x: Week(x//100, x %100).enddate())

df['abbreviation'] = df['geo_value'].apply(lambda x: x.upper())

locations = pd.read_csv('./locations.csv')[['abbreviation', 'location', 'location_name']]

df = df.merge(locations, on='abbreviation')

In [44]:
df

,geo_value,time_value,issue,value,issue_date,target_end_date,abbreviation,location,location_name
0,ak,202540,202547,2.0,2025-11-22,2025-10-04,AK,02,Alaska
1,ak,202540,202548,2.0,2025-11-29,2025-10-04,AK,02,Alaska
2,ak,202540,202549,2.0,2025-12-06,2025-10-04,AK,02,Alaska
3,ak,202540,202550,2.0,2025-12-13,2025-10-04,AK,02,Alaska
4,ak,202540,202551,2.0,2025-12-20,2025-10-04,AK,02,Alaska
...,...,...,...,...,...,...,...,...,...
6196,us,202601,202603,29080.0,2026-01-24,2026-01-10,US,US,US
6197,us,202601,202604,29360.0,2026-01-31,2026-01-10,US,US,US
6198,us,202602,202603,17268.0,2026-01-24,2026-01-17,US,US,US
6199,us,202602,202604,18893.0,2026-01-31,2026-01-17,US,US,US


In [47]:
reference_date = '2026-01-17'
loc = 'US'
last_obs = pd.to_datetime(reference_date) - timedelta(days=7)
    
#obs_vers = df.copy()
obs_vers['target_end_date'] = pd.to_datetime(obs_vers['target_end_date'])
obs_vers['issue_date'] = pd.to_datetime(obs_vers['issue_date'])
obs_subset = obs_vers[(obs_vers.location == loc) &  (obs_vers.target_end_date == last_obs) &\
                      (obs_vers.issue_date==pd.to_datetime(reference_date))]

In [48]:
obs_subset

,geo_value,time_value,issue,value,issue_date,target_end_date,abbreviation,location,location_name


In [ ]:
# previous code

# Get observed value
last_obs = pd.to_datetime(reference_date) - timedelta(days=7)
try:
    obs_date = last_obs.strftime('%Y-%m-%d')
    obs_vers = pd.read_csv(f'https://raw.githubusercontent.com/cdcepi/FluSight-forecast-hub/refs/heads/main/auxiliary-data/target-data-archive/target-hospital-admissions_{obs_date}.csv')
    obs_vers['date'] = pd.to_datetime(obs_vers['date'])
    obs_subset = obs_vers[(obs_vers.location == loc) &  (obs_vers.date == last_obs)]
except:
    obs_subset = obs[(obs.location == loc) &  (obs.date == last_obs)]
